In [1]:
import os

In [2]:
import importlib

In [3]:
import pyiron_gpl

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
from pyiron_base import state, GenericMaster

In [7]:
from pyiron_atomistics import Project

In [8]:
def body_order(n=32, b=5):
    if b == 2:
        return [[i ,n-i] for i in range(n+1)]
    else:
        lst = []
        for i in range(n+1):
            for j in body_order(n=n-i, b=b-1):
                lst.append([i] + j)
        return lst

In [9]:
state.settings.configuration

{'user': 'pyiron',
 'resource_paths': ['/Users/jan/pyiron/resources',
  '/Users/jan/miniforge3/share/pyiron'],
 'project_paths': ['/Users/jan/PycharmProjects/pyiron_mpie/',
  '/Users/jan/pyiron/projects/',
  '/Users/jan/PycharmProjects/pyiron_atomistics/',
  '/Users/jan/PycharmProjects/pyiron_base/'],
 'connection_timeout': 0,
 'sql_connection_string': None,
 'sql_table_name': 'jobs_pyiron',
 'sql_view_connection_string': None,
 'sql_view_table_name': None,
 'sql_view_user': None,
 'sql_view_user_key': None,
 'sql_file': '/Users/jan/pyiron/pyiron.db',
 'sql_host': None,
 'sql_type': 'SQLite',
 'sql_user_key': None,
 'sql_database': None,
 'project_check_enabled': False,
 'disable_database': False,
 'credentials_file': None,
 'write_work_dir_warnings': True,
 'config_file_permissions_warning': True}

In [10]:
project = Project('workflow')

In [11]:
project.remove_jobs(recursive=True, silently=True)

In [12]:
structure = project.create.structure.ase.bulk("Al", cubic=True).repeat([2,2,2])

In [13]:
element_lst = ["Fe", "Ni", "Cr", "Co", "Cu"]

In [14]:
number_atoms = len(structure)

In [15]:
number_species = len(element_lst)

In [16]:
cores = 1

In [17]:
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [18]:
lst_32 = body_order(n=number_atoms, b=number_species)[:10]

In [19]:
structure_collect_lst, elastic_collect_lst = [], []

In [20]:
for l, conc_lst in enumerate(tqdm(lst_32)):
    # Generate structure 
    if sum(np.array(conc_lst)==0) != 4:
        mole_fraction = {
            el: conc/number_atoms
            for el, conc in zip(element_lst, conc_lst)
            if conc > 0
        }
        job = project.create_job(project.job_type.SQSJob, "sqs_" + str(l))
        # job._interactive_disable_log_file = True
        job.structure = structure
        job.input['mole_fractions'] = mole_fraction
        job.input['iterations'] = 1e6
        job.server.cores = cores
        job.run()
        structure_next = job._lst_of_struct[-1]
    else:
        structure_next = structure.copy()
        structure_next.symbols[:] = np.array(element_lst)[np.array(conc_lst)!=0][0] 
    structure_collect_lst.append(structure_next)
    
    # Minimize strucute 
    lmp_mini1 = project.create_job(project.job_type.Lammps, "lmp_mini_" + str(l), delete_existing_job=True)
    lmp_mini1.structure = structure_next
    lmp_mini1.potential = potential
    lmp_mini1.calc_minimize(pressure=0.0)
    # lmp_mini1.server.run_mode.interactive = True
    lmp_mini1._interactive_disable_log_file = True  # disable lammps.log 
    lmp_mini1.run()
    lmp_mini1.interactive_close()

    # Elastic constants 
    lmp_elastic = project.create_job(project.job_type.Lammps, "lmp_elastic_" + str(l), delete_existing_job=True)
    lmp_elastic.structure = lmp_mini1.get_structure()
    lmp_elastic.potential = potential
    lmp_elastic.interactive_enforce_structure_reset = True 
    lmp_elastic.server.run_mode.interactive = True
    # lmp_elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic = lmp_elastic.create_job(project.job_type.ElasticMatrixJob, "elastic_" + str(l), delete_existing_job=True)
    # elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic.run()
    
    elastic_collect_lst.append(elastic._data["C"])

  0%|                                                    | 0/10 [00:00<?, ?it/s]

The job lmp_mini_0 was saved and received the ID: 47
The job elastic_0 was saved and received the ID: 48
The job elastic_0_lmp_elastic_0 was saved and received the ID: 49


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 10%|████▍                                       | 1/10 [00:06<00:58,  6.52s/it]

The job sqs_1 was saved and received the ID: 50
The job lmp_mini_1 was saved and received the ID: 51
The job elastic_1 was saved and received the ID: 52
The job elastic_1_lmp_elastic_1 was saved and received the ID: 53


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 20%|████████▊                                   | 2/10 [00:12<00:51,  6.43s/it]

...

/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 80%|███████████████████████████████████▏        | 8/10 [01:13<00:20, 10.29s/it]

The job sqs_8 was saved and received the ID: 78
The job lmp_mini_8 was saved and received the ID: 79
The job elastic_8 was saved and received the ID: 80
The job elastic_8_lmp_elastic_8 was saved and received the ID: 81


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 90%|███████████████████████████████████████▌    | 9/10 [01:21<00:09,  9.64s/it]

The job sqs_9 was saved and received the ID: 82
The job lmp_mini_9 was saved and received the ID: 83
The job elastic_9 was saved and received the ID: 84
The job elastic_9_lmp_elastic_9 was saved and received the ID: 85


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
100%|███████████████████████████████████████████| 10/10 [01:29<00:00,  8.94s/it]


In [21]:
elastic_collect_lst

[array([[171.550807  , 116.38765601, 116.38765601,   0.        ,
           0.        ,   0.        ],
        [116.38765601, 171.550807  , 116.38765601,   0.        ,
           0.        ,   0.        ],
        [116.38765601, 116.38765601, 171.550807  ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,  76.35007615,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
          76.35007615,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,  76.35007615]]),
 array([[168.20637662, 117.34259605, 117.34259605,   0.        ,
           0.        ,   0.        ],
        [117.34259605, 168.20637662, 117.34259605,   0.        ,
           0.        ,   0.        ],
        [117.34259605, 117.34259605, 168.20637662,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,  78.1351861 ,
      

In [22]:
[len(struct) for struct in structure_collect_lst]

[32, 32, 32, 32, 32, 32, 32, 32, 32, 32]